# Premium Insurance Preprocessing

### Gerekli Kütüphaneleri  ve Veri Setini Yüklemek

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from argparse import Namespace
import seaborn as sns 

from sklearn import preprocessing
from warnings import filterwarnings
filterwarnings('ignore')

sns.set(rc={'figure.figsize':(10,8)})

> Yukarıda gerekli kütüphaneleri import ettik. 
> Özellikle ileriki satırlarımızda göreceğimiz age scale ve bmi scale işlemlerinde kullanmak için "from sklearn.preprocessing import scale" ifadesini import ediyoruz buraya dikkat çekmek istiyorum.
> Kısaca transform işlemimizi gerçekleştirmemiz için gerekli ifadeler olduğunu söyleyebilirim.
> 

In [ ]:
args = Namespace(
    target="expenses",
    data_file="0_dataset.csv",
    cv = 10
)

In [ ]:
premium =  pd.read_csv("../input/insurance-premium-prediction/insurance.csv")
df = premium.copy()

> İlk 10 kayda bakarak veya siz rastgele 10 kayda (df.sample(10)) bakarak da öznitelikleri inceleyebilirsiniz.
;

In [ ]:
df.head(10)


> İnfo komutumuzu da çalıştırıp verimizdeki özniteliklerde null değerimiz var mı varsa kaç tane var ayrıca değişken türlerimizi ve her öznitelikde kaç adet gözlem var görmek istiyorum.

In [ ]:
df.info()

In [ ]:
df.isna().sum()

> Burada güzel bir bilgiyi elde ediyoruz, null bilgimizin olmadığını görüyoruz.
> Fakat kesin emin olmak amacıyla herbir özniteliğimize unique komutunu uygulayıp bilgimizi desteklemek istiyorum .
> Çünkü bazen gözlemi boş bırakmak yerine "" veya boş gibi ifadelerde yer alabiliyor.
>  

In [ ]:
df["age"].unique()

In [ ]:
df["sex"].unique()

In [ ]:
df["bmi"].unique()

In [ ]:
df["children"].unique()

In [ ]:
df["smoker"].unique()

In [ ]:
df["region"].unique()

In [ ]:
df["expenses"].unique()

> Expenses da gözlemlerimizi tamamını göremiyoruz o yüzden df.sample(30) komutunu görmek istiyorum.

In [ ]:
df.sample(30)

> Birkaç defa çalıştırmama rağmen null değer göremiyorum .Null değer olmadığını sayarak devam ediyorum.
> Bunu farklı bi yol ile , 0-100 100-200 200-300 300-400 ... arası değerleri gözlemleyerek de kesinleştirebliriz

In [ ]:
df.describe()

### Ön İşleme

> Ön işleme aşamasında label encoding işlemimizi gerçekleştireceğim.
> "Sex" ve "Smoker" özniteliklerimizde label encoding işlemlerimizi uygulayacağım.
> Çünkü makine öğrenmemiz de kullanmamız için 0 1 formuna transform etmemiz gerekiyor.

#### Label Encoding (Binary)

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
df["sex"] = le.fit_transform(df["sex"])  # 0 is female, 1 is male
df.head()

In [ ]:
df["smoker"] = le.fit_transform(df["smoker"])  # 1 is yes, 0 is no
df.head()

> Tekrar df.head() komutu ile datamın yeni şeklini gördüm.

#### Dummy Variable Extraction (Multi)

> Evet Burada da birbirinden üstünlüğü olmayan ve 2 den fazla unique değere sahip özniteliklerimizle bir işlem gercekleştirip onlarıda sayısal olarak verime eklemek istiyorum .
> Komutumuzu uyguladığımız özniteliğin tüm unique değerleri için birer öznitelik alanı ekleyeceğim ve gözlemimizde önceki özniteliğe ait olan değeri yeni özniteliğinde 1 değerini alacak 

In [ ]:
df_region = pd.get_dummies(df["region"])
df_region.head()

> Ve bu ingilizce yönler yerine bize daha tanıdık gelen şehir isimlerini kullanmak istiyorum.

In [ ]:
df_region.columns = ['Bursa', 'Ankara', 'Istanbul', 'Izmir']
df_region.head()

In [ ]:
df = pd.concat([df, df_region], axis = 1)
df.drop(["Bursa", "region"], axis = 1, inplace = True)
df.head()

> Burda önemli bir değişiklik gerçekleştirdim , eklediğm özniteliklerden birisini drop ettim.
> Bunun ana sebebi "BURSA" olması değildi tabiki ,eğer bir gözlemimizde "ANKARA" "İstanbul" ve "İzmir" gözlemleri 0 ise demekki "BURSA"dan olduğunu anlayacağız.Bunun için ekstra bir öznitelik tutmamıza gerek yok idi bu yüzden drop işlemini rastgele 4 ünden birini seçerek gerçekleştirdim.

## Normalize Data

> Datamızdakş "Age" ve "Bmi"(vücut kitle indeksi) değerlerini 0 - 1 ölceğinde değerlendirmek istiyorum bunu 0= minumum değer ve 1 = maximum değer olarak oranlıyor.

In [ ]:
df.head()

In [ ]:
age = df[['age']].values.astype(float)

min_max_scaler = preprocessing.MinMaxScaler()
age_scaled = min_max_scaler.fit_transform(age)
df['age_scaled'] = pd.DataFrame(age_scaled)

In [ ]:
bmi = df[['bmi']].values.astype(float) 

min_max_scaler = preprocessing.MinMaxScaler() 
bmi_scaled = min_max_scaler.fit_transform(bmi) 
df['bmi_scaled'] = pd.DataFrame(bmi_scaled) 

In [ ]:
df.head()

> 2 yeni öznitelik ekledik bu durumda "Age" ve "Bmi" değerlerimizle işimiz kalmadı bu yüzden drop edip preprocessing işlemimize son vereceğiz.

In [ ]:
df.drop(["age", "bmi"], axis = 1, inplace = True)
df.head()

In [ ]:
export_csv = df.to_csv (r'1_preprocessed_data.csv', index = None, header = True)

**Artık Özel Sigorta data analizimizde export ettiğimiz Data yı kullanacağız.**

***Bir Sonraki Analizimizde Görüşmek Dileğiyle Veriyle Kalın...***